### IEMS 394 PreProcessing

This notebook focuses on organising the preprocessing steps required to ready data for the biofuels optimisation model. For a full description, please refer to *IEMS394: Biofuels Final Report.pdf* which can be found in the corresponding [Biofuels github repo](www.github.com/iems394/blob/master).

In [1]:
%%time
import matplotlib.pyplot as plt
from shapely.geometry import shape, Point, Polygon
import pandas as pd
import numpy as np
from tqdm import tqdm
import os, geojson, multiprocessing, datetime, time, geog, json
import concurrent.futures
# %matplotlib tk

os.chdir('/Users/saifbhatti/Desktop/Northwestern/sy1920/s20/iems394/client-data/cleaned_data')

with open('us_counties_2010.json', encoding = "ISO-8859-1") as f:
    gj = geojson.load(f)
selected_states = ['TX','CA','MN'] #for the purposes of this model, only three states were considered.

CPU times: user 2.57 s, sys: 205 ms, total: 2.77 s
Wall time: 2.65 s


In [2]:
EV_ranges_df =  pd.read_csv('EV_ranges.csv')
EV_ranges_df['Range'].mean()

109.3076923076923

#### Compiling Electric Fueling Stations by county

In [3]:
efuel_stations = pd.read_csv('electric_fuel_stations.csv')
efuel_stations['Points'] = list(zip(efuel_stations['Longitude'], efuel_stations['Latitude']))

In [4]:
def geo_two(x):
    '''
    geo_map takes pandas Series object constructed in the format (longitude, latitude).
    Taking this as the point, a geo.json file is imported containing all US counties from the 2010 census as polygons.
    It computes whether the point is within a polygon, and returns a list of all matches. 
    If a point is not matched, it returns None.
    '''
    counties = []
    for i in tqdm(x):
        point = Point(x)
        for i in range(len(gj['features'])):
            polygon = shape(gj['features'][i]['geometry'])
            if polygon.contains(point):
                counties.append(gj['features'][i]['properties']['NAME'])

In [5]:
# counties = []
# points_list = list(efuel_stations['Points'])
# for i in tqdm(points_list):
#     counties.append(geo_map(i)) #takes 40 minutes to run
# efuel_stations['County'] = counties
# electric_fuel_df.to_csv('electric_fuel_stations.csv')

In [6]:
efuel_stations = efuel_stations[['Station Name','County','City','State','Points']]

#### Converting zipcode-level data to county-level data

In [7]:
tx_ev_vehicle_registered = pd.read_csv('vehicle_reg/tx_ev_registrations_public.csv')
print(len(tx_ev_vehicle_registered['ZIP Code'].unique())) #number of unique zipcodes with registered vehicles
print(len(tx_ev_vehicle_registered)) #number of registrations
# tx_ev_vehicle_registered.head()

1325
74143


In [8]:
mn_ev_vehicle_registered = pd.read_csv('vehicle_reg/mn_ev_registrations_public.csv')
mn_ev_vehicle_registered.drop(['VIN Prefix','VIN Model Year'],inplace=True,axis=1)
print(len(mn_ev_vehicle_registered['ZIP Code'].unique())) #number of unique zipcodes with registered vehicles
print(len(mn_ev_vehicle_registered)) #number of registrations
# mn_ev_vehicle_registered.head()

600
29248


In [9]:
result = pd.concat([mn_ev_vehicle_registered, tx_ev_vehicle_registered], axis=0)
print(len(result['ZIP Code'].unique()))
print(result.shape)  #MN + TX registrations
# result.head()

1906
(103391, 5)


In [59]:
us_zips = pd.read_csv('uszips.csv')
print(us_zips.shape)
print(us_zips.columns)


(33099, 18)
Index(['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'zcta',
       'parent_zcta', 'population', 'density', 'county_fips', 'county_name',
       'county_weights', 'county_names_all', 'county_fips_all', 'imprecise',
       'military', 'timezone'],
      dtype='object')


,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
29993,90001,33.97397,-118.24953,Los Angeles,CA,California,True,NaN,58975,6295.8,6037,Los Angeles,{'06037':100},Los Angeles,06037,False,False,America/Los_Angeles
29994,90002,33.94906,-118.24673,Los Angeles,CA,California,True,NaN,53111,6458.9,6037,Los Angeles,{'06037':100},Los Angeles,06037,False,False,America/Los_Angeles
29995,90003,33.96411,-118.27370,Los Angeles,CA,California,True,NaN,72741,7204.7,6037,Los Angeles,{'06037':100},Los Angeles,06037,False,False,America/Los_Angeles
29996,90004,34.07621,-118.31084,Los Angeles,CA,California,True,NaN,61586,7876.3,6037,Los Angeles,{'06037':100},Los Angeles,06037,False,False,America/Los_Angeles
29997,90005,34.05915,-118.30643,Los Angeles,CA,California,True,NaN,39479,13421.3,6037,Los Angeles,{'06037':100},Los Angeles,06037,False,False,America/Los_Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31749,96146,39.19846,-120.23904,Olympic Valley,CA,California,True,NaN,969,25.2,6061,Placer,{'06061':100},Placer,06061,False,False,America/Los_Angeles
31750,96148,39.24685,-120.05805,Tahoe Vista,CA,California,True,NaN,696,223.8,6061,Placer,{'06061':100},Placer,06061,False,False,America/Los_Angeles
31751,96150,38.87090,-120.00886,South Lake Tahoe,CA,California,True,NaN,29357,93.1,6017,El Dorado,{'06017':100},El Dorado,06017,False,False,America/Los_Angeles
31752,96155,38.75192,-120.09170,South Lake Tahoe,CA,California,True,NaN,0,0.1,6017,El Dorado,{'06017':100},El Dorado,06017,False,False,America/Los_Angeles


In [11]:
combo_zip_county = pd.merge(result, us_zips, how='inner', left_on='ZIP Code', right_on='zip')

In [53]:
print(len(combo_zip_county[(combo_zip_county['state_id'] == 'TX') | (combo_zip_county['state_id'] == 'MN')])) #how many within MN & TX
combo_zip_county = combo_zip_county[(combo_zip_county['state_id'] == 'TX') | (combo_zip_county['state_id'] == 'MN')]
print(combo_zip_county['state_id'].unique())
# new.to_csv('ev_registration_county.csv')


97888
['MN' 'TX']


,DMV ID,ZIP Code,Registration Expiration Date,Registration Valid Date,Vehicle Name,zip,lat,lng,city,state_id,...,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,6,55347.0,01/12/2020,30/01/2020,NaN,55347,44.82932,-93.46632,Eden Prairie,MN,...,30390,687.4,27053,Hennepin,{'27053':100},Hennepin,27053,False,False,America/Chicago
1,6,55347.0,01/12/2020,16/01/2020,NaN,55347,44.82932,-93.46632,Eden Prairie,MN,...,30390,687.4,27053,Hennepin,{'27053':100},Hennepin,27053,False,False,America/Chicago
2,6,55347.0,01/10/2020,09/01/2020,NaN,55347,44.82932,-93.46632,Eden Prairie,MN,...,30390,687.4,27053,Hennepin,{'27053':100},Hennepin,27053,False,False,America/Chicago
3,6,55347.0,01/12/2020,06/01/2020,NaN,55347,44.82932,-93.46632,Eden Prairie,MN,...,30390,687.4,27053,Hennepin,{'27053':100},Hennepin,27053,False,False,America/Chicago
4,6,55347.0,01/12/2020,03/01/2020,NaN,55347,44.82932,-93.46632,Eden Prairie,MN,...,30390,687.4,27053,Hennepin,{'27053':100},Hennepin,27053,False,False,America/Chicago
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101263,3,79325.0,NaN,03/09/2019,Ford C-Max Energi,79325,34.41493,-102.94000,Farwell,TX,...,2361,5.0,48369,Parmer,"{'48017':0.99,'48369':99.01}",Parmer|Bailey,48369|48017,False,False,America/Chicago
101264,3,79339.0,NaN,03/09/2019,Nissan Leaf,79339,33.88987,-102.33544,Littlefield,TX,...,6658,10.1,48279,Lamb,"{'48219':1.78,'48279':98.22}",Lamb|Hockley,48279|48219,False,False,America/Chicago
101265,3,79504.0,NaN,03/09/2019,Ford Fusion Energi,79504,32.32056,-99.35231,Baird,TX,...,3204,2.9,48059,Callahan,"{'48059':99.57,'48417':0.43}",Callahan|Shackelford,48059|48417,False,False,America/Chicago
101266,3,79772.0,NaN,03/09/2019,Tesla Model 3,79772,31.42539,-103.62296,Pecos,TX,...,13875,4.3,48389,Reeves,"{'48109':0,'48389':99.72,'48475':0.28}",Reeves|Ward|Culberson,48389|48475|48109,False,False,America/Chicago


In [19]:
cali = pd.read_csv('california_car_data.csv')
print(cali.columns)
cali

Index(['Date', 'Zip Code', 'Model Year', 'Fuel', 'Make', 'Duty', 'Vehicles'], dtype='object')


/Users/saifbhatti/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,10/1/2018,90000,2006,Gasoline,OTHER/UNK,Light,1
1,10/1/2018,90000,2014,Gasoline,OTHER/UNK,Light,1
2,10/1/2018,90000,2016,Gasoline,OTHER/UNK,Light,1
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Light,1
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,55
...,...,...,...,...,...,...,...
586228,10/1/2018,Other,<2006,Hybrid Gasoline,TOYOTA,Light,54
586229,10/1/2018,Other,<2006,Natural Gas,OTHER/UNK,Heavy,4
586230,10/1/2018,Other,<2006,Natural Gas,OTHER/UNK,Light,6
586231,10/1/2018,Other,<2006,Other,OTHER/UNK,Heavy,1


#### IRS / Median income data [source]??

#### E85 Fueling Stations and Viability Index Computations

In [23]:
e85 = pd.read_csv('e85_fuel_stations.csv')
e85['Points'] = list(zip(e85['Longitude'], e85['Latitude']))
e85 = e85[['Station Name','City','County', 'State','ZIP','Points']]
e85.groupby('County').count().sort_values(by='City', ascending=False).head(5)

,Station Name,City,State,ZIP,Points
County,,,,,
Montgomery,56,56,56,56,56
Hennepin,54,54,54,54,54
Polk,47,47,47,47,47
Harris,47,47,47,47,47
Cook,45,45,45,45,45


In [21]:
# counties = []
# points_list = list(e85['Points'])
# for i in tqdm(points_list): 
#     counties.append(geo_map(i))
# e85['County'] = counties
# e85.to_csv('e85_fuel_stations.csv')

In [15]:
centroids = pd.read_csv('county_centroids.csv')
#cleaning
centroids['Latitude'] = centroids['Latitude'].apply(lambda x: x.strip('°'))
centroids['Longitude'] = centroids['Longitude'].apply(lambda x: x.strip('°'))
centroids['Longitude'] = centroids['Longitude'].apply(lambda x: x.replace('–','-'))
centroids['Latitude'] = centroids['Latitude'].apply(lambda x: x.replace('+',''))

In [16]:
centroids['Points'] = list(zip(centroids['Longitude'], centroids['Latitude']))
selected_centroids = centroids[centroids['State'].isin(selected_states)]

In [17]:
def create_circles(x,rad):
    '''
    county_scanner takes pandas Series object constructed in the format (longitude, latitude).
    Taking a point (which is the fuelling station) and constructs a circle of radius 'rad'.
    This is stored to a geojson file.???
    '''
    point = Point(x)
    return point.buffer(rad)    

In [18]:
e85_points = e85['Points']
circles = []
for i in tqdm(e85_points):
    circles.append(create_circles(i,10000))

100%|██████████| 3561/3561 [00:00<00:00, 8182.39it/s]


- Some county name recurr within the set of states we have selected. Such counties have had their respective 2-letter state code appended to their name. They are enumerated below.
    1. Orange - CA,TX 
    2. Cass - MN, TX
    3. Lake - CA, MN
    4. Trinity - CA, TX
    5. Houston - MN, TX
    6. Polk - MN,TX
    7. Brown - MN,TX
    8. Clay - MN, TX
    9. Jackson - TX,MN
    10. Washington - MN, TX
    11. Martin - MN,TX

In [117]:
repeated_list = ['Orange','Cass','Lake','Trinity','Houston','Polk','Brown','Clay','Jackson','Washington','Martin']

def county_renaming_engine(df,repeated_list):
    '''
    pd.DataFrame -> renamed pd.DataFrame
    '''
    df.loc[tester['County'].isin(repeated_list), 'County'] = df['County'] + '_' + df['State']
    return

In [108]:
tester = pd.read_csv('https://raw.githubusercontent.com/saif1457/iems394/master/data/testing%20county_renaming_engine.csv')
# tester.set_index(tester['County'],inplace=True)
# tester.index.where(repeated_list)
tester[tester['County'].isin(repeated_list)]

tester.loc[tester['County'].isin(repeated_list), 'County'] = tester['County'] + '_' + tester['State']


In [122]:
e85
df2 = e85.filter(regex='County|county|State|state_id')
df2
county_renaming_engine(df2,repeated_list)
# spike_cols = [col for col in e85.columns if 'County' in col | if 'county' in col]   
# spike_cols

# import pandas as pd

# data = {'spike-2': [1,2,3], 'hey spke': [4,5,6], 'spiked-in': [7,8,9], 'no': [10,11,12]}
# df = pd.DataFrame(data)

# spike_cols = [col for col in df.columns if 'spike' in col]
# print(list(df.columns))
# print(spike_cols)
# df2 = df.filter(regex='spike')
# print(df2)

IndexingError: (0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
Name: County, dtype: bool, 'County')

In [76]:
zip_county = combo_zip_county[combo_zip_county['state_id'].isin(selected_states)]

In [77]:
zip_county['state_id'].unique()

array(['MN', 'TX'], dtype=object)

In [96]:
cali_vehicle_data = us_zips.merge(cali,right_on='Zip Code',left_on='zip').groupby('county_name').sum()
cali_vehicle_data.reset_index(inplace=True)
cali_vehicle_data = cali_vehicle_data[['county_name','population','Vehicles']]
cali_vehicle_data.to_csv('2018cali_vehicle_data.csv')

In [88]:
cali

,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,10/1/2018,90000,2006,Gasoline,OTHER/UNK,Light,1
1,10/1/2018,90000,2014,Gasoline,OTHER/UNK,Light,1
2,10/1/2018,90000,2016,Gasoline,OTHER/UNK,Light,1
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Light,1
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,55
...,...,...,...,...,...,...,...
586228,10/1/2018,Other,<2006,Hybrid Gasoline,TOYOTA,Light,54
586229,10/1/2018,Other,<2006,Natural Gas,OTHER/UNK,Heavy,4
586230,10/1/2018,Other,<2006,Natural Gas,OTHER/UNK,Light,6
586231,10/1/2018,Other,<2006,Other,OTHER/UNK,Heavy,1


{'total_vehicles_registered': {'Alameda': 1217269,
  'Alpine': 476,
  'Amador': 1697,
  'Butte': 89172,
  'Calaveras': 55341,
  'Colusa': 8682,
  'Contra Costa': 917891,
  'Del Norte': 20285,
  'El Dorado': 22323,
  'Fresno': 689723,
  'Glenn': 11399,
  'Humboldt': 115509,
  'Imperial': 160949,
  'Inyo': 20806,
  'Kern': 616616,
  'Kings': 95132,
  'Lake_CA': 68357,
  'Lassen': 14144,
  'Los Angeles': 7391982,
  'Madera': 113016,
  'Marin': 218822,
  'Mariposa': 19670,
  'Mendocino': 89279,
  'Merced': 197104,
  'Modoc': 3926,
  'Mono': 11887,
  'Monterey': 328583,
  'Napa': 124367,
  'Nevada': 40032,
  'Orange_CA': 2555482,
  'Placer': 52565,
  'Plumas': 8109,
  'Riverside': 1761050,
  'Sacramento': 292136,
  'San Benito': 52934,
  'San Bernardino': 1596698,
  'San Diego': 2597948,
  'San Francisco': 454533,
  'San Joaquin': 559680,
  'San Luis Obispo': 239665,
  'San Mateo': 716318,
  'Santa Barbara': 344180,
  'Santa Clara': 1496123,
  'Santa Cruz': 237902,
  'Shasta': 71833,
  'Sie

In [148]:
T.dtypes

county                       object
total_vehicles_registered     int64
dtype: object